In [1]:
from typing import List, Dict
import json
from pathlib import Path
from fontTools import ttLib
from fontTools.ttLib.tables._c_m_a_p import CmapSubtable
from fontTools.pens.ttGlyphPen import TTGlyphPen
from fontTools import svgLib

In [2]:
path_info = Path('svgs/info.json')
path_save = Path('fonts/test.ttf')

In [3]:
for ext in ['.ttf', '.otf', '.woff']:
    path_example = Path('fonts/arkham-symbols').with_suffix(ext)
    fonts_ex = ttLib.TTFont(
        path_example, lazy=False,
        sfntVersion="\000\001\000\000" if ext=='.ttf' else "OTTO",
        flavor = 'woff' if ext=='.woff' else None
    )
    cmap = fonts_ex.getBestCmap()
    print(path_example)
    print(cmap)
    print(fonts_ex.keys())
    print()

fonts\arkham-symbols.ttf
{49: 'nc', 50: 'hw', 51: 'wh', 52: 'jf', 53: 'sc', 54: 'par', 65: 'rtnotz', 66: 'rtdl', 67: 'rtptc', 68: 'rtfa', 69: 'rtcu', 72: 'eoec', 97: 'core', 98: 'tdl', 99: 'tptc', 100: 'tfa', 101: 'tcu', 102: 'tde', 103: 'tic', 104: 'eoep'}
['GlyphOrder', 'head', 'hhea', 'maxp', 'OS/2', 'hmtx', 'cmap', 'cvt ', 'loca', 'glyf', 'name', 'post', 'gasp', 'FFTM']

fonts\arkham-symbols.otf
{49: 'nc', 50: 'hw', 51: 'wh', 52: 'jf', 53: 'sc', 54: 'par', 65: 'rtnotz', 66: 'rtdl', 67: 'rtptc', 68: 'rtfa', 69: 'rtcu', 72: 'eoec', 97: 'core', 98: 'tdl', 99: 'tptc', 100: 'tfa', 101: 'tcu', 102: 'tde', 103: 'tic', 104: 'eoep'}
['GlyphOrder', 'head', 'hhea', 'maxp', 'OS/2', 'name', 'cmap', 'post', 'CFF ', 'FFTM', 'hmtx']

fonts\arkham-symbols.woff
{49: 'nc', 50: 'hw', 51: 'wh', 52: 'jf', 53: 'sc', 54: 'par', 65: 'rtnotz', 66: 'rtdl', 67: 'rtptc', 68: 'rtfa', 69: 'rtcu', 72: 'eoec', 97: 'core', 98: 'tdl', 99: 'tptc', 100: 'tfa', 101: 'tcu', 102: 'tde', 103: 'tic', 104: 'eoep'}
['GlyphOr

In [52]:
path_example = Path('fonts/arkham-symbols').with_suffix('.ttf')
fonts_ex = ttLib.TTFont(
    path_example, lazy=False,
    sfntVersion="\000\001\000\000" if ext=='.ttf' else "OTTO",
    flavor = 'woff' if ext=='.woff' else None
)
table = fonts_ex['glyf']
print(table.glyphOrder)
print(table.glyphs)

['.notdef', '.null', 'nonmarkingreturn', 'nc', 'hw', 'wh', 'jf', 'sc', 'par', 'rtnotz', 'rtdl', 'rtptc', 'rtfa', 'rtcu', 'eoec', 'core', 'tdl', 'tptc', 'tfa', 'tcu', 'tde', 'tic', 'eoep']
{'.notdef': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47490>, '.null': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47400>, 'nonmarkingreturn': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB473D0>, 'nc': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47370>, 'hw': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47250>, 'wh': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47220>, 'jf': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB45DB0>, 'sc': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB47AC0>, 'par': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB475B0>, 'rtnotz': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22CB45120>, 'rtdl': <fontTools.ttLib

In [66]:
# fonts_ex['maxp'].recalc(fonts_ex)
for name, glyph in table.glyphs.items():
    try:
        print(name, glyph.xMin, glyph.xMax, glyph.yMin, glyph.yMax, sep='\t')
    except:
        print(name, '\t\tNULL')

.notdef	33	298	0	666
.null 		NULL
nonmarkingreturn 		NULL
nc	-1	901	-202	801
hw	-1	1205	-209	804
wh	-4	878	-203	803
jf	-1	1322	-206	801
sc	0	1005	-200	803
par	181	718	-93	702
rtnotz	1	999	-200	800
rtdl	0	1000	-200	800
rtptc	0	1000	-201	801
rtfa	0	1000	-200	800
rtcu	69	942	-108	758
eoec	116	916	-27	748
core	8	991	-200	801
tdl	0	1000	-188	788
tptc	153	849	-201	800
tfa	-1	1001	-130	742
tcu	-1	1003	33	586
tde	0	1001	-167	773
tic	146	854	-200	800
eoep	84	888	-118	721


In [4]:
with path_info.open(encoding='utf-8') as filept:
    data: List[Dict[str, str]] = json.load(filept)
print(data[0])

{'char': 'a', 'name': 'core', 'path': '01_NZ.svg'}


In [22]:
maxp = ttLib.newTable('maxp')
loca = ttLib.newTable('loca')
ccf = ttLib.newTable('CCF ')
ccf2 = ttLib.newTable('CCF2')
head = ttLib.newTable('head')


In [28]:
# truetype
sfntVersion = "\000\001\000\000"
flavor = None

font = ttLib.TTFont(sfntVersion=sfntVersion, flavor=flavor, lazy=False)
for name in ['glyf', 'hmtx', 'name', 'cmap', 'maxp', 'head', 'maxp', 'head']:
    font[name] = ttLib.newTable(name)

In [29]:
cmap = font['cmap']
cmap.tableVersion = 0
cmap.tables = []
for info in data:
    cmap_ = CmapSubtable.newSubtable(12)
    cmap_.platformID = 0
    cmap_.platEncID = 4
    cmap_.language = 0
    cmap_.cmap = {ord(info['char']): info['name']}
    
    cmap.tables.append(cmap_)

In [30]:
name = font['name']
name.setName("ⓒFantasy Flight Publishing, Inc and 2023 (주)코리아보드게임즈. All Right Reserved.", 0, 0, 4, 0)
name.setName("arkham symbols", 1, 0, 4, 0)
name.setName("regular", 2, 0, 4, 0)
name.setName("Version 1.0 Feb 24 2023", 5, 0, 4, 0)
name.setName("This font is the merge from the Arkham horror: Card Game and is a trademark of Fantasy Flight Games.", 7, 0, 4, 0)

In [39]:
glyf = font['glyf']
glyf.glyphs = {}
glyf.glyphOrder = []
hmtx = font['hmtx']
hmtx.metrics = {}

for info in data:
    name = info['name']
    path = path_info.parent / info['path']
    svg = svgLib.path.SVGPath(path)
    pen = TTGlyphPen({ord(info['char']): name})
    svg.draw(pen)
    glyph = pen.glyph()
    
    hmtx[name] = (1000, 0)
    glyf[name] = glyph
font.setGlyphOrder(glyf.glyphOrder)

In [54]:
print(glyf.glyphs)
print(glyf.glyphOrder)

{'core': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C536EF0>, 'rtnotz': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C535390>, 'tdl': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C5356F0>, 'rtdl': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74B5E0>, 'tptc': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C749120>, 'rtptc': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74BA90>, 'tfa': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74B7F0>, 'rtfa': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74B850>, 'tcu': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74B010>, 'rtcu': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74B4F0>, 'tde': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C7493C0>, 'tic': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001E22C74AE60>, 'eoep': <fontTools.ttLib.tables._g_l_y_f.Glyph object at 0x000001

In [55]:
maxp = font['maxp']
maxp.numGlyphs = len(glyf.glyphOrder)
maxp.tableVersion = 0x00010000
maxp.recalc(font)

AttributeError: 'Glyph' object has no attribute 'xMin'